In [1]:
#Importação das bibliotecas mencionadas no readme.md
import cv2
import numpy as np

In [2]:
'''
BAIXAR https://pjreddie.com/media/files/yolov3.weights !!!
Arquivo muito grande para o github.
'''

'\nBAIXAR https://pjreddie.com/media/files/yolov3.weights !!!\nArquivo muito grande para o github.\n'

In [3]:
'''o modelo YOLOv3 pré-treinado (pesos e arquivos de configuração)
https://github.com/pjreddie/darknet/blob/master/data/coco.names -> arquivo contendo a lista de classes, cada linha representa uma classe

https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg -> arquivo de configuração do modelo

https://pjreddie.com/media/files/yolov3.weights -> arquivo de pesos'''

# Carregar as configurações e pesos do modelo YOLOv3 pré-treinado para detecção de objetos
net = cv2.dnn.readNet("../../yolov3.weights", "./arquivos_config/yolov3.cfg")

In [4]:
# Carregar lista de classes que o modelo YOLOv3 foi treinado para reconhecer (cada linha representa uma classe)
classes = []
with open("./arquivos_config/coco.names", "r") as f:
    classes = f.read().strip().split("\n")

In [5]:
# Carregar a imagem para teste
image = cv2.imread("./imagens/imagem2.png")
image_original = cv2.imread("./imagens/imagem2.png")

In [6]:
# Obter as camadas de saída da rede YOLOv3
output_layers = net.getUnconnectedOutLayersNames()

In [7]:
# Redimensionar a imagem e normalizar os valores dos pixels
# (pré-processar a imagem para criar um blob de imagem que pode ser alimentado para a rede neural)
blob = cv2.dnn.blobFromImage(image, scalefactor=0.00392, size=(416, 416), swapRB=True, crop=False)

In [8]:
# Passar a imagem pela rede YOLOv3 para detecção de objetos
net.setInput(blob) # para obter as detecções
outs = net.forward(output_layers) # camadas de saída da rede

In [9]:
# Processar as detecções e mostrar na imagem
class_ids = []
confidences = []
boxes = []

#Analise as detecções retornadas pelas camadas de saída da rede.
#A classe com a maior pontuação é selecionada como a classe identificada para cada detecção.
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:  # Defina um limiar de confiança
            # Coordenadas da caixa delimitadora
            center_x = int(detection[0] * image.shape[1])
            center_y = int(detection[1] * image.shape[0])
            w = int(detection[2] * image.shape[1])
            h = int(detection[3] * image.shape[0])

            # Coordenadas do canto superior esquerdo
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)


In [10]:
# Para eliminar detecções redundantes
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4) # aplicando a supressão não máxima às caixas delimitadoras

font = cv2.FONT_HERSHEY_PLAIN

# Utilizando as informações das detecções para desenhar caixas delimitadoras e exibir o nome da classe e a confiança associada.
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence = confidences[i]
        color = (255, 0, 0)
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image, f"{label}", (x, y - 5), font, 1, color, 1)
        #cv2.putText(image, f"{label} {confidence:.2f}", (x, y - 5), font, 1, color, 1)


In [11]:
# Mostrar a imagem resultante com as detecções desenhadas
cv2.imshow("Deteccao de Residuos", image)
cv2.imshow("Imagem original", image_original)
cv2.waitKey(0)
cv2.destroyAllWindows()
